# OraculumLocalBox setup

There are two way to setup and experiment with a local installation of Oraculum and Sibylla. This interactive notebook is designed to play and experiment with the .NET API.

## Prerequisites

In order to deploy _Oraculum_ locally you need to install _Docker_ (on Windows using the WSL2).

## Keys

In this notebook we reference _oraculum.secret.conf_ and _weaviate.secret.yml_. Those are copies of _oraculum.conf_ and _weaviate.yml_ where you should indicate you OpenAI org id and key. You can get those from the [OpenAI dashboard](https://platform.openai.com/account/api-keys).

## First run

The initial deployment of _Oraculum_ requires the schema initialization first, then the Facts data structure can be populated. Follow the steps (in some cases a timeout may require re-evaluate the cell, or even restarting the kernels if stuck in evaluation):

In [10]:
# Start the _Weaviate_
docker-compose -f .\weaviate.secret.yml up -d

 Container weaviate  Starting
 Container weaviate  Started


Check that Weaviate is up and running by pointing your browser to http://localhost:8080/v1/swagger and, in case you are prompted for credentials, access with empty credentials.

### Schema initialization

First you must reference the _Oraculum_ nuget package:

In [1]:
#r "nuget:Oraculum"

Installed Packages Oraculum, 1.0.2

It is suggested to always test wether the schema has already been intialized (otherwise the data will be lost):

In [3]:
using Oraculum;

var oraculumConf = Oraculum.Configuration.FromJson(System.IO.File.ReadAllText("oraculum.secret.conf"));

var oraculum = new Oraculum.Oraculum(oraculumConf);

if (!await oraculum.IsKBInitialized()) {
    Console.WriteLine("Initializing KB");
    await oraculum.Init();
} else {
    Console.WriteLine("KB already initialized");
}

await oraculum.Connect();

Initializing KB


## Fact managment

Oraculum API for managing facts is straightforward, you can get facts filtering by `catergory`, by `tags` and other attributes. The `FindRelevantFacts` method allows to query the facts database obtaining results ordered by vector distance of the embeddings.

### Populate the facts collection

_Weaviate_ supports bulk upload of data so it is preferable to add an array of facts instead of iterating and add individual facts. In any case is the `AddFact` method that is overloaded to support both scenarios.

We populate the facts by reading Q&A from a JSON file (taken from [here](https://www.watercoolertrivia.com/trivia-questions/movie-trivia-questions)):

In [4]:
if (await oraculum.TotalFactsByCategory("movie") == 0) {
    Console.WriteLine("Loading KB");
    var jsonqa = System.Text.Json.JsonDocument.Parse(System.IO.File.ReadAllText("qamoviedb.json"));
    var i = 0;
    var l = new List<Fact>();
    foreach (var o in jsonqa.RootElement.EnumerateArray()) {
        i++;
        var q =o.GetProperty("q").GetString();
        var a =o.GetProperty("a").GetString();
        var f = new Fact() {
            category = "movie",
            citation = $"FAQ #{i}",
            title = q,
            content = a,
            factType = "faq"
        };
        l.Add(f);
    }
    await oraculum.AddFact(l);
    Console.WriteLine($"KB loaded (#{await oraculum.TotalFacts()})");
} else {
    Console.WriteLine($"KB already loaded ({await oraculum.TotalFacts()} facts)");
}



Loading KB
KB loaded (#287)


## Testing Sibylla

The code for running Sibylla can be found in the `NewSibyllaSession.cs` cmdlet in the `OraculumCLI` project. The code is pretty straightforward and at its core is like this:

    do
    {
        this.CommandRuntime.Host.UI.Write("You: ");
        var user = this.CommandRuntime.Host.UI.ReadLine();
        if (user == "#quit")
            break;
        this.CommandRuntime.Host.UI.Write(ConsoleColor.Yellow, ConsoleColor.Black, $"Assistant: ");
        var ena = sibylla.AnswerAsync(user);
        var en = ena.GetAsyncEnumerator();
        while(true)
        {
            var j = en.MoveNextAsync();
            j.AsTask().Wait();
            if (!j.Result)
                break;
            this.CommandRuntime.Host.UI.Write(ConsoleColor.Yellow, ConsoleColor.Black, $"{en.Current}");
        }
        this.CommandRuntime.Host.UI.WriteLine();
    } while (true);

The _Oraculum_ kernel for .NET interactive performs similarly and will be used in this notebook to test the assistant.

The first step is to reference the _OraculumInteractive_ nuget package and load the _Oraculum_ kernel:

In [6]:
#r "nuget:OraculumInteractive"

Installed Packages OraculumInteractive, 1.0.0

In [7]:
OraculumInteractive.OraculumKernelExtension.LoadOraculum(Microsoft.DotNet.Interactive.KernelInvocationContext.Current.HandlingKernel.RootKernel);

Oraculum kernel loaded

Now we can create a Sibylla configuration. It is an important data structure because it defines the assistant behavior. In particular it states that the knowledge will be added in the XML format:

In [8]:
#!value --name SibyllaConfig 
{"BaseSystemPrompt":"You are a passionate assistant with knowledge about movies and you will answer to users about movies curiosities. You answer only to questions about movies and curiosities, to other curiosities you shall answer \"I am authorized only to answer to questions about movies\". To answer you will only use ground truth and information about facts in XML that will follow instead of your knowledge. If you use information from an XML having a 'cit' attribute include the citation in square brakets in the output.",
"BaseAssistantPrompt":"Hello from your personal movie assistant. I know curiosities about 287 movies, try me!",
"MaxTokens":1024,
"Model":"gpt-3.5-turbo",
"Temperature":0.1,
"TopP":1,
"FrequencyPenalty":0,
"PresencePenalty":0,
"FactFilter": null,
"CategoryFilter": ["movie"],
"TagFilter": null
}

Before testing Sibylla you need to connect the kernel to the Sibylla based on the `SibyllaConfig` configuration:

In [9]:
#!connect Oraculum --kernel-name SibyllaMovie --oraculum-config .\oraculum.conf --sibylla-config-json @value:SibyllaConfig

Kernel added: #!SibyllaMovie

Once the kernel has been instatiated you can change the code cell type to SibyllaMovie and ask questions! The FAQ is used when cited as FAQ#_n_ where _n_ is the index of the question in the FAQ file.

In [10]:
What do you know about Star Wars?

I know a lot about Star Wars! It is a popular film franchise that began in 1977 with the release of "Star Wars: Episode IV - A New Hope." The franchise was created by George Lucas and has since expanded to include multiple trilogies, spin-off films, TV series, books, and merchandise.

One interesting fact is that Daisy Ridley made her first appearance as Rey in "Star Wars: Episode VII - The Force Awakens" [FAQ #39]. Another curiosity is that in "Star Wars: Episode II - Attack of the Clones," Anakin Skywalker tells Padme Amidala that he doesn't like sand because "it's coarse and rough and irritating, and it gets everywhere" [FAQ #41].

The Rebel Alliance in the Star Wars films uses a starfighter called the X-Wing Starfighter, which has four laser cannons and two proton torpedo launchers [FAQ #37]. Additionally, the 1998 disaster film "Armageddon" references Star Wars, with Ben Affleck's character declaring, "I'm Han and you're Chewbacca" [FAQ #201].

Lastly, the 2018 Star Wars film "Sol

In [11]:
Tell me something interesting about the first trilogy!

The first trilogy, also known as the original trilogy, is comprised of "Star Wars: Episode IV - A New Hope" (1977), "Star Wars: Episode V - The Empire Strikes Back" (1980), and "Star Wars: Episode VI - Return of the Jedi" (1983). Here are a few interesting facts about the original trilogy:

1. The first Star Wars film, "A New Hope," was initially released simply as "Star Wars" and became a massive success, leading to the creation of the entire franchise.

2. "The Empire Strikes Back" is often regarded as one of the best sequels ever made and is known for its iconic twist involving the character of Darth Vader.

3. The character of Yoda, the wise and powerful Jedi Master, made his first appearance in "The Empire Strikes Back." Yoda's unique way of speaking, with his sentences often structured in an unconventional manner, has become one of his defining characteristics.

4. "Return of the Jedi" features the climactic battle on the forest moon of Endor, where the Rebel Alliance takes on th

In [12]:
What actor played President George W. Bush in a 2008 film?

Josh Brolin played President George W. Bush in the 2008 film "W." [FAQ #17]